# Yiddish TTS — Colab setup

This notebook sets up a Colab environment that can run the `generate_yiddish_speech.py` script from your repository.

Follow cells in order. If you use GPU, switch Runtime -> Change runtime type -> GPU. For Coqui TTS compatibility, CPU runtime is acceptable but slower.

# Yiddish TTS — Colab setup

This notebook sets up a Colab environment that can run the `generate_yiddish_speech.py` script from your repository.

Follow cells in order. If you use GPU, switch Runtime -> Change runtime type -> GPU. For Coqui TTS compatibility, CPU runtime is acceptable but slower.

## 1) Mount Google Drive

Mount Drive so you can persist outputs and upload your repository ZIP into Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Create a working folder in Drive for this notebook
import os
workdir = '/content/drive/MyDrive/yiddish_tts_colab'
os.makedirs(workdir, exist_ok=True)
print('Working directory:', workdir)
print('Drive mounted; you can upload your repo ZIP to this folder and unzip it here.')
:
,
: {
:
 },
: [
2
,
,

:
,
: {
:
 },
: [
,
,
,
,
,

## 3) Important: Python version compatibility

Colab may ship newer system Python (for example 3.13). Coqui TTS and many audio libraries currently require Python <= 3.11. Two options are provided below:

1) Quick (may fail on 3.13): install packages into the notebook environment with pip.
2) Robust (recommended): install a separate Python 3.11 runtime using micromamba/conda and use that Python to install TTS.

### 3a) Quick install (use only if your Colab runtime is <= 3.11)

Run this cell to install required packages into the current Python. If you see errors related to Python version incompatibility, skip to the micromamba section.

In [ ]:
# Quick install into the current runtime (may fail on Python 3.13)
!pip install --upgrade pip setuptools wheel
!pip install --quiet --no-cache-dir torch==2.2.2+cpu --index-url https://download.pytorch.org/whl/cpu || true
!pip install --quiet --no-cache-dir TTS soundfile librosa numpy==1.24.3 pandas scipy matplotlib scikit-learn PyYAML tqdm tensorboard psutil || true
print('Quick install attempted (errors above if any).')

### 3b) Robust (recommended) — create Python 3.11 using micromamba and install packages there

This creates an isolated conda-style environment with Python 3.11, installs TTS and dependencies, and then runs the repository scripts with that Python binary. This avoids conflicts with Colab's system Python.

In [ ]:
# Install micromamba (small, fast) and create a Python 3.11 environment
import os, textwrap
MAMBA_DIR = '/content/micromamba'
ENV_PREFIX = '/content/py311'
if not os.path.exists(MAMBA_DIR):
    print('Installing micromamba...')
    !wget -qO micromamba.tar.bz2 https://micromamba.snakepit.net/api/micromamba/linux-64/latest
    !mkdir -p {MAMBA_DIR} && tar -xjf micromamba.tar.bz2 -C {MAMBA_DIR} --strip-components=1
else:
    print('micromamba already downloaded')
# Create env with python 3.11 + pip
if not os.path.exists(ENV_PREFIX):
    print('Creating Python 3.11 environment... (this may take a minute)')
    !{MAMBA_DIR}/bin/micromamba create -y -p {ENV_PREFIX} python=3.11 pip -c conda-forge
else:
    print('Python 3.11 environment already exists at', ENV_PREFIX)
# Show the python binary path to use
python_bin = os.path.join(ENV_PREFIX, 'bin', 'python')
print('Use this python for installs and running the repo:', python_bin)
print('
Now install TTS and audio deps into that Python (next cell).')

In [ ]:
# Install Coqui TTS and required packages into the micromamba env
from pathlib import Path
PY311 = Path('/content/py311/bin/python')
if not PY311.exists():
    raise SystemExit('Python 3.11 env not found. Run the cell that creates micromamba env first.')
# Upgrade pip inside the env and install packages (adjust pins if needed)
!/content/py311/bin/python -m pip install --upgrade pip setuptools wheel
!/content/py311/bin/python -m pip install --no-cache-dir --index-url https://download.pytorch.org/whl/cpu torch==2.2.2+cpu || true
!/content/py311/bin/python -m pip install --no-cache-dir TTS==0.22.0 soundfile librosa numpy==1.24.3 pandas==2.2.2 scipy matplotlib scikit-learn PyYAML tqdm tensorboard psutil || true
print('Installation into Python 3.11 env completed (or attempted).')

## 4) Import Libraries & Reproducibility

Import main libraries and set seeds for reproducibility. If you used the micromamba path, run these checks using the created Python binary (examples below).

In [ ]:
# If you installed into the micromamba env, run the check with that python:
# /content/py311/bin/python -c "import torch, sys; print(sys.version); print(torch.__version__, torch.cuda.is_available())"

# For small checks in the current notebook (may still be the system python):
import random, numpy as np, os, sys
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
try:
    import torch
    torch.manual_seed(SEED)
    print('Torch version (current runtime):', torch.__version__)
except Exception as e:
    print('Torch not available in this runtime:', e)
print('Current Python:', sys.version)
print('If you want to run the repo using Python 3.11, use /content/py311/bin/python')
:
,
: {
:
 },
: [
5
,
,

:
,
: {
:
 },
: [
,
,
,
,

In [ ]:
# Example: unzip a repo ZIP placed in Drive working dir
import os
repo_zip = os.path.join(workdir, 'yiddish_TTS_improved.zip')
if os.path.exists(repo_zip):
    print('Found ZIP, extracting...')
    !unzip -q "$repo_zip" -d /content/yiddish_repo
    print('Extracted to /content/yiddish_repo')
else:
    print('No ZIP found at', repo_zip)


## 6) Data Loading and Preprocessing (reference audio check)

Ensure your dataset reference audio exists or upload a replacement file.

In [ ]:
# Check for the default reference audio path used by the script
default_ref = '/content/yiddish_repo/tts_segments/audio/segment_0001.wav'
import os
if os.path.exists(default_ref):
    print('Reference audio found:', default_ref)
else:
    print('Reference audio not found at default location.')
    print('If you have a local reference audio, upload it now:')
    # files.upload()  # uncomment to interactively upload a reference audio file

## 7) Core functions — run the `generate_yiddish_speech.py` script

This cell runs the repository script with a custom Yiddish phrase and shows output. Adjust paths if you unzipped to a different folder. If you used micromamba, call the /content/py311/bin/python binary.

In [ ]:
# Path to the repo script
script_path = '/content/yiddish_repo/generate_yiddish_speech.py'
if not os.path.exists(script_path):
    # Fallback to searching current working dir if you uploaded just the script
    script_path = '/content/generate_yiddish_speech.py'

if not os.path.exists(script_path):
    raise FileNotFoundError('generate_yiddish_speech.py not found in notebook. Upload it or unzip repo into Drive and rerun this cell.')

# Example run with a custom Yiddish phrase
yiddish_text = 'שבת שלום, ווי גייט עס?'
output_file = '/content/yiddish_output.wav'
print('Running script with system python (if you want to use Python 3.11 env, replace python3 with /content/py311/bin/python)')
!python3 "$script_path" "$yiddish_text" || true
print('If the script ran, output saved to', output_file, 'or look in the repo-specific sample folder.')

In [ ]:
# Run the script without args to generate built-in sample phrases (no-arg mode)
!python3 /content/yiddish_repo/generate_yiddish_speech.py || true
print('Script executed (errors shown above if any). Check yiddish_speech_samples/ in the repo folder for outputs.')
:
,
: {
:
 },
: [
8
,
,

:
,
: {
:
 },
: [
!cp -r /content/yiddish_repo/yiddish_speech_samples /content/drive/MyDrive/yiddish_tts_colab/outputs || true
print('Copied any generated samples to Drive: /content/drive/MyDrive/yiddish_tts_colab/outputs')

## 9) Basic runtime checks

A few asserts to validate the environment and basic functions. If you used the micromamba flow, run the checks with /content/py311/bin/python.

In [ ]:
# Check that TTS can be imported in the current runtime (or use the py311 python to test the env)
try:
    from TTS.api import TTS
    print('TTS import OK (current runtime)')
except Exception as e:
    print('TTS import failed in current runtime:', e)
# Check torch import and device
import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

## 10) Visualize results

You can listen to output files or plot spectrograms. Example to display an audio player for an output file.

In [ ]:
from IPython.display import Audio, display
example_out = '/content/yiddish_output.wav'
import os
if os.path.exists(example_out):
    display(Audio(example_out))
else:
    print('No example output available at', example_out)